In [1]:
import numpy as np
import random as rn
import findspark
import sys
import time
import numba
from numba import cuda, float32, float64
findspark.init("/opt/spark")

In [2]:
from pyspark import SparkContext
sc = SparkContext("local","PSO Distribuido")

In [3]:
def MSE(y, pred):
    n = len(y)
    if n != len(pred):
        print("error: datos y predicción de distintos tamaños")
        return -1

    resultado = 0.0

    for i in range(n):
        resultado += (y[i] - pred[i])**2

    resultado = resultado / n

    return resultado

In [4]:
V_max= 5.0
W= 1.0
c_1 = 0.8
c_2 = 0.2
objetivo =[50,50,50]
ind=0
filas_x=[0,0,0]

In [5]:
#Número dimensiones de los vectores
n=3
#Número de partículas
m= 10
#Número de iteraciones
I = 100
posiciones_=[]
mejor_posiciones_locales_=[]
velocidades_=[]
particulas = np.array([posiciones_,velocidades_,mejor_posiciones_locales_])
mejor_pos_global_=[]
mejor_pos_global_arr= np.array([0 for j in range(n)])
best_local_fitness_= []
best_local_fitness_arr=np.array(best_local_fitness_)
#maximum float
max = sys.float_info.max
best_global_fitness = max
accum=sc.accumulator(best_global_fitness)
mejor_pos_global= sc.broadcast(mejor_pos_global_arr)
best_local_fitness= sc.broadcast(best_local_fitness_arr)

In [6]:
def InitParticles(part,N,M,bgf,blf):
    #global best_global_fitness
    for j in range(M):
        posicion=[rn.uniform(-100,100) for j in range(N)]
        posiciones_.append(posicion)
        posiciones= np.array(posiciones_)
        mejor_posiciones_locales_=posiciones_
        mejor_posiciones_locales=np.array(mejor_posiciones_locales_)
        velocidad=[rn.uniform(-5,5) for j in range(N)]
        velocidades_.append(velocidad)
        velocidades= np.array(velocidades_)
        fit = MSE(posiciones[j],objetivo)
        best_local_fitness_.append(fit)
        blf= np.array(best_local_fitness_)
        #print("bgf-> ",bgf)
        if fit < bgf:
            bgf = fit
            accum= fit
            mejor_pos_global = mejor_posiciones_locales[j][:]
    part= np.array([posiciones,velocidades,mejor_posiciones_locales])
    return blf,bgf,mejor_pos_global,part 

In [7]:
def InitParticles2(part,N,M,bgf,blf):
    #global best_global_fitness
    parts_ =[]
    for j in range(M):
        posicion=[rn.uniform(-100,100) for j in range(N)]
        velocidad=[rn.uniform(-5,5) for j in range(N)]
        part_ = posicion + velocidad + posicion
        parts_.append(part_)
        
        

        fit = MSE(posicion,objetivo)
        best_local_fitness_.append(fit)
        blf= np.array(best_local_fitness_)
        #print("bgf-> ",bgf)
        if fit < bgf:
            bgf = fit
            accum= fit
            mejor_pos_global = posicion
    parts = np.array(parts_)
    return blf,bgf,mejor_pos_global,parts 

In [8]:
def fitnessEval(part,mpg,blf,bgf, ind):
    filas=part[0:1,ind]
    fit = MSE(filas[0],objetivo)
    if fit< blf[ind]:
        blf[ind] = fit
        #mpl[ind] = part[0:1,ind]
        part[2:,ind]=filas
        if fit < bgf:
            bgf = fit
            mpg = filas[0]
            accum= fit
    return part,mpg,blf,bgf

In [9]:
#ind=sc.accumulator(0)
def fitnessEval_x(part,mpg,blf,bgf):
    global ind
    filas=part[0:1]
    fit = MSE(filas[0],objetivo)
    if fit < blf[ind]:
        blf[ind] = fit
        #mpl[ind] = part[0:1,ind]
        part[2:]=filas
        if fit < bgf:
            bgf = fit
            mpg = filas[0]
            accum.value= fit
    if ind<10:
        ind+=1
    return part,mpg,blf,bgf

In [10]:
def fitnessEval_x2(part,mpg,blf,bgf,N):
    global ind
    filas=part[0:N]
    fit = MSE(filas,objetivo)
    if fit < blf[ind]:
        blf[ind] = fit
        #mpl[ind] = part[0:1,ind]
        for k in range(N):
            part[2*N+k]=filas[k]
        if fit < bgf:
            bgf = fit
            mpg = filas
            accum= fit
    if ind<10:
        ind+=1
    #return part,mpg,blf,bgf
    return part

In [11]:
#def posEval(part,mpl,mpg,ind):
def posEval(part,mpg,ind,N):
    velocidades=part[1:2,ind][0]
    mpl=part[2:,ind][0]
    for k in range(N):
        velocidades[k] = W*velocidades[k] + c_1*r_1*(mpl[k] - part[0:1,ind][0][k]) + c_2*r_2*(mpg[k] - part[0:1,ind][0][k])
        if velocidades[k] > V_max:
            velocidades[k] = V_max
        elif velocidades[k] < -V_max:
            velocidades[k] = -V_max  
        part[0:1,ind][0][k] = part[0:1,ind][0][k] + velocidades[k]
        part[1:2,ind][0][k] = velocidades[k]
    #return part,velocidades
    return part

In [12]:
#def posEval(part,mpl,mpg,ind):
def posEval_x(part,mpg,N):
#def posEval_x(part,N):
    global ind
    velocidades=part[1:2][0]
    mpl=part[2:][0]
    r_1 = rn.random()
    r_2 = rn.random()
    for k in range(N):
        velocidades[k] = W*velocidades[k] + c_1*r_1*(mpl[k] - part[0:1][0][k]) + c_2*r_2*(mpg[k] - part[0:1][0][k])
        if velocidades[k] > V_max:
            velocidades[k] = V_max
        elif velocidades[k] < -V_max:
            velocidades[k] = -V_max  
        part[0:1][0][k] = part[0:1][0][k] + velocidades[k]
        part[1:2][0][k] = velocidades[k]
    #return part,velocidades
    return part

In [13]:
def posEval_x2(part,mpg,N):
#def posEval_x(part,N):
    global ind
    velocidades=part[N:2*N]
    mpl=part[2*N:]
    r_1 = rn.random()
    r_2 = rn.random()
    for k in range(N):
        velocidades[k] = W*velocidades[k] + c_1*r_1*(mpl[k] - part[k]) + c_2*r_2*(mpg[k] - part[k])
        if velocidades[k] > V_max:
            velocidades[k] = V_max
        elif velocidades[k] < -V_max:
            velocidades[k] = -V_max  
        part[k] = part[k] + velocidades[k]
        part[N+k] = velocidades[k]
    #return part,velocidades
    return part

In [14]:
def coresPerSM():
    cc_cores_per_SM_dict = {
    (2,0) : 32,
    (2,1) : 48,
    (3,0) : 192,
    (3,5) : 192,
    (3,7) : 192,
    (5,0) : 128,
    (5,2) : 128,
    (6,0) : 64,
    (6,1) : 128,
    (7,0) : 64,
    (7,5) : 64,
    (8,0) : 64,
    (8,6) : 128,
    (8,9) : 128,
    (9,0) : 128
    }
    # the above dictionary should result in a value of "None" if a cc match 
    # is not found.  The dictionary needs to be extended as new devices become
    # available, and currently does not account for all Jetson devices
    device = cuda.get_current_device()
    my_sms = getattr(device, 'MULTIPROCESSOR_COUNT')
    my_cc = device.compute_capability
    cores_per_sm = cc_cores_per_SM_dict.get(my_cc)
    total_cores = cores_per_sm*my_sms
    print("GPU compute capability: " , my_cc)
    print("GPU total number of SMs: " , my_sms)
    print("total cores: " , total_cores)
    return cores_per_sm

In [15]:
best_local_fitness,best_global_fitness,mejor_pos_global,particulas=InitParticles2(particulas,n,m,best_global_fitness,best_local_fitness)
print("particulas:", particulas)
#print("particulas[0:1]-> ",particulas[0:1][0])
#print("mejor velocidades-> ",particulas[1:2][0])
#print("mejor posiciones locales-> ",particulas[2:][0])
#print("mejor_pos_global-> ",mejor_pos_global)
#print("best_global_fitness",best_global_fitness)
#print("best_local_fitness",best_local_fitness)
print("device =", cuda.get_current_device())
print("cores/SM= ",coresPerSM())

particulas: [[  1.6648522   15.10927671 -47.6512364    1.21019216  -2.97485968
    3.70558615   1.6648522   15.10927671 -47.6512364 ]
 [ 58.75170428  32.70181433  -5.08752188  -4.87211616   1.36884471
    4.38583173  58.75170428  32.70181433  -5.08752188]
 [-35.61878216  22.03611845  91.15520929  -2.55024224  -0.23377097
    2.46210682 -35.61878216  22.03611845  91.15520929]
 [ 73.94380845 -51.25686552 -57.17564748  -4.19169408  -1.37767547
   -1.49712906  73.94380845 -51.25686552 -57.17564748]
 [ 62.03035568  65.29922443 -23.2372825   -4.72561126   0.33932543
   -1.56283511  62.03035568  65.29922443 -23.2372825 ]
 [-17.87595551   3.47947027 -63.6127141    0.70565621  -4.34191539
   -3.34125475 -17.87595551   3.47947027 -63.6127141 ]
 [-42.09449538 -35.34238277  41.01234143  -0.48098083   3.09725549
   -0.26157353 -42.09449538 -35.34238277  41.01234143]
 [ 72.9097306   34.19108893 -51.98961925  -2.08966622   1.72805537
   -4.47436264  72.9097306   34.19108893 -51.98961925]
 [ 66.217262

In [16]:
rdd_master= sc.parallelize(particulas)
rdd_prueba = rdd_master.map(lambda x:x[0])
rdd_prueba = rdd_prueba.collect()
print(rdd_prueba)

[1.6648521984256206, 58.751704278132735, -35.61878215840851, 73.94380845022133, 62.03035568295189, -17.875955513574326, -42.0944953800342, 72.90973059936141, 66.2172620940361, 68.55614338147595]


In [18]:
rdd_master= sc.parallelize(particulas)
#print("particulas-> ",particulas[0:1])
#print("velocidades-> ",particulas[1:2])
#print("mejores posiciones locales-> ",particulas[2:])
start = time.time()

print("rdd_master antes:", rdd_master.collect())
for i in range(100):
    #print("antes:",mejor_pos_global)
    rdd_2 = rdd_master.map(lambda x:fitnessEval_x2(x,mejor_pos_global,best_local_fitness,best_global_fitness,n))
    #print("rdd_2:", rdd_2.collect())
    #print("despues:",mejor_pos_global)
    resultado = rdd_2.map(lambda x:posEval_x2(x,mejor_pos_global,n))
    resultado = resultado.collect()
    
    for j in range(m):
        particulas[j] = resultado[j][:]
        
    #print("resultado:", resultado[0])
    rdd_master= sc.parallelize(particulas)
    #rdd_master = rdd_3
#rdd_master.map(lambda x:posEval_x(x,mejor_pos_global,n))
    #particulas[0:1]=resultado[0]
    #particulas[1:2]=resultado[1]
    #particulas[2:3]=resultado[2]
print("rdd_master despues:", rdd_master.collect())
    
#print("particulas despues:", particulas)
    
    #rdd_master = rdd_master.collect()
    #print("rdd_master:", rdd_master)
#print("posiciones-> ",particulas[0:1])
#print("velocidades-> ",particulas[1:2])
#print("mejores posiciones locales-> ",particulas[2:])
rdd_master= sc.parallelize(particulas)
   # resultado = resultado_2
end = time.time()
tiempo = end - start
resultado = rdd_master.collect()
#prueba = prueba.collect()
#print("resultado-> ",resultado)
#print("prueba-> ",prueba)
print("Tiempo de ejecucion(s): ", tiempo)
#print(resultado)
#print("particulas-> ",particulas[0:1][0])
print("mejor_pos_global-> ",mejor_pos_global)
print("mejor fitness global-> ", accum.value, MSE(mejor_pos_global,objetivo))

rdd_master antes: [array([ 72.24682054,  20.42332417, -17.06679608,   1.85352931,
        -4.34606332,  -4.37516707,  70.39329123,  24.76938749,
       -12.69162901]), array([50.32250217, 36.06803671,  3.27428928, -0.72593066,  0.29120302,
        0.72096333, 51.04843283, 35.77683369,  2.55332594]), array([51.97535019, 16.05464751,  2.46972297,  2.31992602,  0.98369412,
       -2.32883778, 49.65542417, 15.0709534 ,  4.79856075]), array([ 65.12874438,  47.39840821, -14.68842823,  -1.82207269,
         1.60413522,  -4.12104112,  66.95081707,  45.794273  ,
       -10.56738711]), array([38.90682925, 53.94914678, -8.14405156, -2.89835023,  1.93770116,
        1.14135803, 41.80517948, 52.01144562, -9.2854096 ]), array([ 51.36936762,  45.83889812, -13.95010639,   3.37572591,
        -2.06438431,   3.19307142,  47.9936417 ,  47.90328243,
       -17.14317782]), array([ 47.21221015,  45.72352672, -17.70820485,   0.59586144,
         2.02378197,  -0.3624762 ,  46.61634871,  43.69974475,
       -1